# 实验一 数据探索与预处理

## 1 数据清洗

### 1.1 找出不符合规则的数据并进行修正

In [1]:
import pandas as pd
import numpy as np

# 读取csv
# 中文解码这个我也不知道怎么高效处理，一般试试utf-8再试试gbk，再不行就搜索报错信息...
# 长号码串不读取为数字更好处理
data = pd.read_csv("ex1.csv", encoding='gb18030', dtype={'qq':object, 'tel':object})

In [2]:
# 对于每一列,对空白字符进行处理
data["姓名"].replace("\s+", " ", regex=True, inplace=True)
data["公司名称"].replace("\s+", "", regex=True, inplace=True)
data["工作内容"].replace("\s+", "", regex=True, inplace=True)
data["email"].replace("\s+", "", regex=True, inplace=True)
data["注册资金"].replace("\s+", "", regex=True, inplace=True)
data["教育经历"].replace("\s+", " ", regex=True, inplace=True) 

# 姓名列只允许中文英文
data["姓名"].replace("[^\w\u4e00-\u9fff]+", np.NaN, regex=True, inplace=True)

# Tel列只允许合法格式
data["tel"].replace("^[^1[35678]\d{9}$]", np.NaN, regex=True, inplace=True)

# 去除姓名字段为空的行
data.dropna(subset=["姓名"], inplace=True)


In [3]:
data

,是否认证,姓名,部门-职务,公司名称,工作内容,qq,email,tel,公司是否认证,注册资金,公司成立时间,公司年龄,教育经历,工作经历
0,1,陈文才,客户经理,重庆心联宇科技有限公司,NaN,NaN,NaN,NaN,1,200万,2017/1/20,1年,重庆科技学院 1 热能与动力工程 2012-08-01 2016-06-01,重庆心联宇科技有限公司
1,1,黄彬,总部 执行董事,重庆善古堂科技有限公司,NaN,NaN,NaN,NaN,1,100万,2009/5/25,9年,第一军医大学 1 临床医学系 1990-01-01 1993-01-01,重庆善古堂科技有限公司 总部 执行董事 2008-05-01 现在
2,1,李玉萍,短信营销部 销售经理,北京亿美软通科技有限公司广州分公司,短信群发推广，短信验证码,NaN,NaN,NaN,1,NaN,2010/9/27,7年,广东农工商职业技术学院 0 市场营销 2014-09-01 2017-06-01,北京亿美软通科技有限公司广州分公司 短信营销部 销售经理
3,1,黄勇钦,市场部 大客户销售经理,重庆亿邦运输有限责任公司,川渝地区的仓储配送业务以及西南地区网络招商加盟。,NaN,NaN,NaN,1,200万,2009/1/7,9年,NaN,重庆亿邦运输有限责任公司
4,1,王旺,市场部 销售经理,深圳市深米网络科技有限公司,极速上线小程序,NaN,NaN,NaN,1,1000万,2014/5/30,4年,NaN,深圳深米网络科技有限公司
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7479,1,吕斌,总经办 总经理,宁波市众协国际货运代理有限公司,公司整体经营运作,NaN,NaN,NaN,1,500万,2009/2/13,9年,NaN,NaN
7480,1,吉凯凯,360销售事业部 网络运营,北京全时天地在线网络信息股份有限公司,网络运营,NaN,NaN,NaN,1,4500万,2005/12/30,12年,NaN,北京全时天地在线网络信息股份有限公司 360销售事业部 网络运营
7481,1,吴巧龙,置业顾问,北京首开万科房地产开发有限公司,NaN,NaN,NaN,NaN,1,1亿,2013/3/8,5年,河北地质大学 1 工商管理 2011-09-01 2015-06-01,北京首开万科房地产开发有限公司 置业顾问
7482,0,客户经理,客户经理,上海易谷网络科技有限公司,NaN,NaN,NaN,NaN,1,5000万,2008/1/31,10年,NaN,上海易谷网络科技有限公司 大客户经理


### 1.2 根据字段含义将数据处理为合适的格式

In [4]:
# 拆分部门-职务

# 直接拆分成两个新列
d1 = data["部门-职务"].str.split(" ", expand=True)
d1.drop(d1.columns[2:], axis=1, inplace=True)
d1.columns = ["部门", "职务"]

# 删除旧列, 添加新列
data = data.drop("部门-职务", axis=1).join(d1)
data

,是否认证,姓名,公司名称,工作内容,qq,email,tel,公司是否认证,注册资金,公司成立时间,公司年龄,教育经历,工作经历,部门,职务
0,1,陈文才,重庆心联宇科技有限公司,NaN,NaN,NaN,NaN,1,200万,2017/1/20,1年,重庆科技学院 1 热能与动力工程 2012-08-01 2016-06-01,重庆心联宇科技有限公司,,客户经理
1,1,黄彬,重庆善古堂科技有限公司,NaN,NaN,NaN,NaN,1,100万,2009/5/25,9年,第一军医大学 1 临床医学系 1990-01-01 1993-01-01,重庆善古堂科技有限公司 总部 执行董事 2008-05-01 现在,总部,执行董事
2,1,李玉萍,北京亿美软通科技有限公司广州分公司,短信群发推广，短信验证码,NaN,NaN,NaN,1,NaN,2010/9/27,7年,广东农工商职业技术学院 0 市场营销 2014-09-01 2017-06-01,北京亿美软通科技有限公司广州分公司 短信营销部 销售经理,短信营销部,销售经理
3,1,黄勇钦,重庆亿邦运输有限责任公司,川渝地区的仓储配送业务以及西南地区网络招商加盟。,NaN,NaN,NaN,1,200万,2009/1/7,9年,NaN,重庆亿邦运输有限责任公司,市场部,大客户销售经理
4,1,王旺,深圳市深米网络科技有限公司,极速上线小程序,NaN,NaN,NaN,1,1000万,2014/5/30,4年,NaN,深圳深米网络科技有限公司,市场部,销售经理
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7479,1,吕斌,宁波市众协国际货运代理有限公司,公司整体经营运作,NaN,NaN,NaN,1,500万,2009/2/13,9年,NaN,NaN,总经办,总经理
7480,1,吉凯凯,北京全时天地在线网络信息股份有限公司,网络运营,NaN,NaN,NaN,1,4500万,2005/12/30,12年,NaN,北京全时天地在线网络信息股份有限公司 360销售事业部 网络运营,360销售事业部,网络运营
7481,1,吴巧龙,北京首开万科房地产开发有限公司,NaN,NaN,NaN,NaN,1,1亿,2013/3/8,5年,河北地质大学 1 工商管理 2011-09-01 2015-06-01,北京首开万科房地产开发有限公司 置业顾问,,置业顾问
7482,0,客户经理,上海易谷网络科技有限公司,NaN,NaN,NaN,NaN,1,5000万,2008/1/31,10年,NaN,上海易谷网络科技有限公司 大客户经理,,客户经理


In [5]:
# 注册资金转为数值


In [6]:
# 公司成立时间转为yyyy-mm-dd格式

# 公司年龄转数值(计算)

In [7]:
# 教育经历拆分
# 拆分为 学校 是否本科 入学时间 毕业时间

In [8]:
# 工作经历拆分

### 1.3	进行数据概化

In [9]:
# 职务等级划分

In [10]:
# 工作类别划分

In [11]:
# 公司地域划分

# 公司领域划分

In [12]:
# 注册资金等级划分

## 2 数据可视化

### 2.1 按照数据类型进行图表展示

### 2.2 相关分析

### 2.3 对比分析

## 3 思考

### 3.1 数据隐私保护